In [1]:
import numpy as np
from astropy.io import fits
import pandas as pd


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/nasekhian/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/var/folders/xm/sl735y1n4m30kfvcxl7h4p6w0000gn/T/ipykernel_10480/1874662340.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
fits_file = fits.open('seg_22450.fits')

# Extract the data
data = fits_file[0].data

# Close the FITS file

fits_file.close()

In [3]:
# Find the coordinates where both x and y values are zero
zero_coords = np.argwhere(data == 0)

# Filter the coordinates within the range [100, 550]
filtered_coords = [coord for coord in zero_coords if (100 <= coord[1] <= 550) and (100 <= coord[0] <= 550)]

# Convert to a numpy array
filtered_coords_array = np.array(filtered_coords)

# Set the random seed for reproducibility
np.random.seed(42)

# Randomly select 100 coordinates
random_coords_indices = np.random.choice(filtered_coords_array.shape[0], 150, replace=False)
random_coords = filtered_coords_array[random_coords_indices]

# Create a DataFrame from the numpy array with columns 'x0' and 'y0'
coordinates = pd.DataFrame(random_coords, columns=['x0', 'y0'])

# Reset the index to start from 1
coordinates.index = range(1, len(coordinates) + 1)

# Display the DataFrame
print(coordinates)

# Save the DataFrame to a CSV file
coordinates.to_csv('coordinates.csv')

      x0   y0
1    443  526
2    403  121
3    141  415
4    417  404
5    219  143
..   ...  ...
146  160  456
147  477  122
148  214  255
149  280  362
150  234  247

[150 rows x 2 columns]


In [6]:
import numpy as np
from astropy.io import fits
import pandas as pd

def is_circle_empty(data, center, radius):
    y, x = np.ogrid[-center[1]:data.shape[0]-center[1], -center[0]:data.shape[1]-center[0]]
    mask = x*x + y*y <= radius*radius
    return np.all(data[mask] == 0)

# Open the FITS file
fits_file = fits.open('seg_22450.fits')

# Extract the data
data = fits_file[0].data

# Close the FITS file
fits_file.close()

# Initialize variables
radius = 10
num_coordinates = 120
coordinates = []

# Set the random seed for reproducibility
np.random.seed(42)

# Generate coordinates
while len(coordinates) < num_coordinates:
    # Randomly select a coordinate
    coord = np.random.randint(100, data.shape[0] - 100), np.random.randint(100, data.shape[1] - 100)
    
    # Check if the circular area around the coordinate is empty
    if is_circle_empty(data, coord, radius):
        # Add the coordinate to the list
        coordinates.append(coord)

        # Exclude coordinates within the circular area for subsequent selections
        y, x = np.ogrid[-coord[1]:data.shape[0]-coord[1], -coord[0]:data.shape[1]-coord[0]]
        mask = x*x + y*y <= radius*radius
        data[mask] = 1

# Create a DataFrame from the numpy array with columns 'x0' and 'y0'
coordinates2_df = pd.DataFrame(coordinates, columns=['x0', 'y0'])

# Reset the index to start from 1
coordinates2_df.index = range(1, len(coordinates2_df) + 1)

# Display the DataFrame
print(coordinates2_df)

# Save the DataFrame to a CSV file
coordinates2_df.to_csv('coordinates2.csv')


      x0   y0
1    202  535
2    370  206
3    171  120
4    221  566
5    314  430
..   ...  ...
116  599  610
117  400  231
118  514  544
119  190  630
120  138  225

[120 rows x 2 columns]
